In [1]:
import os
from dotenv import load_dotenv

In [2]:
profile = {
    'name' : 'John',
    'full_name' : 'John Doe',
    "user_profile_background": "Senior software engineer leading a team of 5 developers",
}

In [3]:
prompt_instructions = {
    "triage_rules": {
        "ignore": "Marketing newsletters, spam emails, mass company announcements",
        "notify": "Team member out sick, build system notifications, project status updates",
        "respond": "Direct questions from team members, meeting requests, critical bug reports",
    },
    "agent_instructions": "Use these tools when appropriate to help manage John's tasks efficiently."
}

In [4]:
# Example incoming email
email = {
    "from": "Alice Smith <alice.smith@company.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "body": """
Hi John,

I was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?

Specifically, I'm looking at:
- /auth/refresh
- /auth/validate

Thanks!
Alice""",
}

#### Define the first part of the agent - triage

In [5]:
from pydantic import BaseModel, Field
from typing_extensions import TypedDict, Literal, Annotated
from langchain.chat_models import init_chat_model

In [17]:
llm = init_chat_model(
    model = "MFDoom/deepseek-r1-tool-calling:8b",
    model_provider = "ollama", 
    temperature = 0
)

In [10]:
class Router(BaseModel):
    """Analyze the unread email and route it according to its content."""
    reasoning: str = Field(
        description = "Step-by-step reasoning behind the classification"
    )
    
    classification: Literal["ignore", "notify", "respond"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, "
        "'notify' for important information that doesn't need a response, "
        "'respond' for emails that need a reply"
    )

In [18]:
llm_router = llm.with_structured_output(Router)

In [ ]:
from prompts import triage_system_prompt, triage_user_prompt

In [14]:
system_prompt = triage_system_prompt.format(
    full_name=profile["full_name"],
    name=profile["name"],
    examples=None,
    user_profile_background=profile["user_profile_background"],
    triage_no=prompt_instructions["triage_rules"]["ignore"],
    triage_notify=prompt_instructions["triage_rules"]["notify"],
    triage_email=prompt_instructions["triage_rules"]["respond"],
)

In [15]:
user_prompt = triage_user_prompt.format(
    author=email["from"],
    to=email["to"],
    subject=email["subject"],
    email_thread=email["body"],
)

In [19]:
result = llm_router.invoke(
    [
        {'role' : 'system', 'content' : system_prompt},
        {'role' : 'user', 'content' : user_prompt}
    ]
)

In [22]:
print(result)

reasoning="The email is from Alice Smith asking for clarification on missing API documentation endpoints. This involves a direct question from a team member, which requires a response to ensure the information is addressed properly. Therefore, this email should be classified as 'respond' so John can provide the necessary details or further investigation if needed." classification='respond'


#### Main agent, define tools

In [23]:
from langchain_core.tools import tool

In [24]:
# Demo purposes, could use email API to send email

@tool
def write_email(to: str, subject: str, body: str) -> str:
    """Write and send an email"""
    return f"Email sent to {to} with subject '{subject}'"

In [25]:
@tool
def schedule_meeting(
    attendees: list[str],
    subject: str,
    duration_minutes: str,
    preferred_day: str
) -> str:
    """Schedule a calendar meeting."""
    # Placeholder response - in real app would check calendar and schedule
    return f"Meeting '{subject}' scheduled for {preferred_day} with {len(attendees)} attendees"

In [26]:
@tool
def check_calendar_availability(day: str) -> str:
    """Check calendar availability for a given day."""
    # Placeholder response - in real app would check actual calendar
    return f"Available times on {day}: 9:00 AM, 2:00 PM, 4:00 PM"

#### Main agent, define prompt

In [29]:
from prompts import agent_system_prompt

def create_prompt(state):
    return [
        {
            "role" : "system",
            "content" : agent_system_prompt.format(
                instructions=prompt_instructions["agent_instructions"],
                **profile
            )
        }
    ] + state['messages']

In [32]:
from langgraph.prebuilt import create_react_agent
from langchain_ollama import ChatOllama

In [31]:
tools = [write_email, schedule_meeting, check_calendar_availability]

In [ ]:
model = ChatOllama(model = "MFDoom/deepseek-r1-tool-calling:8b")

agent = create_react_agent(
    model = model,
    tools = tools,
    prompt = create_prompt
)